<a href="https://colab.research.google.com/github/VickkiMars/SentOrd/blob/main/SentOrd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers sentencepiece

In [ ]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
import random
import nltk
nltk.download("punkt_tab")
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def make_reorder_example(batch):
    input_texts, target_texts = [], []

    for article in batch["article"]:
        sentences = sent_tokenize(article)

        # Skip bad examples
        if 3 <= len(sentences) <= 20:
            shuffled = sentences.copy()
            random.shuffle(shuffled)

            input_texts.append(" ".join(shuffled))
            target_texts.append(" ".join(sentences))

    # Always return lists, even if empty
    return {
        "input_text": input_texts,
        "target_text": target_texts
    }

In [ ]:
train_data = dataset["train"].map(
    make_reorder_example,
    batched=True,
    remove_columns=["article", "highlights", "id"]
)

val_data = dataset["validation"].map(
    make_reorder_example,
    batched=True,
    remove_columns=["article", "highlights", "id"]
)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5Tokenizer

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

max_input_length = 512
max_target_length = 512

def preprocess(batch):
  inputs = tokenizer(batch["input_text"],
                     padding="max_length",
                     truncation=True,
                     max_length=max_input_length)
  targets = tokenizer(text_target=batch["target_text"], # Use text_target
                      padding="max_length",
                      truncation=True,
                      max_length=max_target_length)
  return {
      "input_ids": inputs.input_ids,
      "attention_mask": inputs.attention_mask,
      "labels": targets.input_ids
  }

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
train_dataset = train_data.map(preprocess, batched=True)
val_datast = val_data.map(preprocess, batched=True)

Map:   0%|          | 0/57103 [00:00<?, ? examples/s]

Map:   0%|          | 0/3777 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0].keys())
# should show: dict_keys(['input_ids', 'attention_mask', 'labels'])

print(train_dataset[0]["input_ids"][:20])  # see first 20 tokens
print(train_dataset[0]["labels"][:20])     # target tokens


dict_keys(['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'])
[3559, 343, 28989, 4425, 32, 7, 9859, 12, 48, 934, 5, 216, 47, 92, 16, 4370, 1054, 16, 8, 907]
[10842, 5577, 3221, 6, 18898, 41, 254, 17235, 61, 1636, 71, 843, 16054, 17604, 11, 3, 14165, 23, 3268, 45]


In [ ]:
from transformers import T5ForConditionalGeneration

model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=3e-5,
    save_total_limit=2,
    fp16=True,
    report_to="none"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.select(range(10000)),
    eval_dataset=val_datast.select(range(2000))
)

In [ ]:
trainer.train()

Step,Training Loss
100,2.994100
200,0.654200
300,0.538600
400,0.441700
500,0.390000
600,0.393500
700,0.347100
800,0.352900
900,0.374200
1000,0.423200


In [ ]:
def reorder_sentences(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Pick one validation example
example = val_datast[10]
print("Shuffled:", example["input_text"])    # scrambled input
print("Target:", example["target_text"])     # original order
print("Predicted:", reorder_sentences(example["input_text"]))  # model output


Shuffled: The attack happened at about 6 p.m. in the Bolan area of Lashkar Gah city, said Mohammad Jan Rasoolyar, deputy governor of Helmand. (CNN)A suicide attacker detonated a car bomb near a police vehicle in the capital of southern Afghanistan's Helmand province on Tuesday, killing seven people and injuring 23 others, the province's deputy governor said. Details about the attacker's identity and motive weren't immediately available. Several children were among the wounded, and the majority of casualties were civilians, Rasoolyar said.
Target: (CNN)A suicide attacker detonated a car bomb near a police vehicle in the capital of southern Afghanistan's Helmand province on Tuesday, killing seven people and injuring 23 others, the province's deputy governor said. The attack happened at about 6 p.m. in the Bolan area of Lashkar Gah city, said Mohammad Jan Rasoolyar, deputy governor of Helmand. Several children were among the wounded, and the majority of casualties were civilians, Rasoolya

In [ ]:
model.save_pretrained("/content/drive/MyDrive/sentence_reorder_t5")
tokenizer.save_pretrained("/content/drive/MyDrive/sentence_reorder_t5")


('/content/drive/MyDrive/sentence_reorder_t5/tokenizer_config.json',
 '/content/drive/MyDrive/sentence_reorder_t5/special_tokens_map.json',
 '/content/drive/MyDrive/sentence_reorder_t5/spiece.model',
 '/content/drive/MyDrive/sentence_reorder_t5/added_tokens.json')